# Polkadot Price Prediction Using **sklearn**

In this project I'll be working with csv taken from [subscan](https://polkadot.subscan.io/tools/charts). I use 8 files, listed below with a brief description:

**Daily Active Account & Newly Created Account.csv**:  the daily active accounts and new accounts created.

**Daily Bond Value (DOT).csv**: daily amount (in dot) bonded for staking.

**Daily Fees Used.csv**: daily amount of fees (in dot) spent on the network.

**Daily Transaction Amount.csv**: amount transacted on the network.

**Daily Unbonding Schedule (DOT).csv**: dot scheduled to be released from staking.

**DOT Price (USD).csv**: the price in USD of 1 dot.

**Treasury Available Income.csv**: available income of treasury (in dot).

**Treasury Expenses.csv**: expenses per day of treasury.


## import packages and csv files

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
active_inactive = pd.read_csv("data/Daily Active Account & Newly Created Account.csv", parse_dates=['Date'])
bond = pd.read_csv("data/Daily Bond Value (DOT).csv", parse_dates=['Date'])
fees = pd.read_csv("data/Daily Fees Used.csv", parse_dates=['Date'])
transactions = pd.read_csv("data/Daily Transaction Amount.csv", parse_dates=['Date'])
unbonding = pd.read_csv("data/Daily Unbonding Schedule (DOT).csv", parse_dates=['Date'])
price = pd.read_csv("data/DOT Price (USD).csv", parse_dates=['Date'])
treasury_available = pd.read_csv('data/Treasury Available Income.csv', parse_dates=['Date'])
treasury_expense = pd.read_csv('data/Treasury Expenses.csv', parse_dates=['Date'])

## rename columns

In [3]:
active_inactive = active_inactive.rename(columns={'Active Account': 'Active_account', 'New Account': 'New_account'})
bond = bond.rename(columns={'Value':'Bond_value'})
fees = fees.rename(columns={'DOT':'DOT_fees'})
transactions = transactions.rename(columns={'DOT':'DOT_transactions'})
unbonding = unbonding.rename(columns={'Value':'Unbonding_value'})
price = price.rename(columns={' Price (USD)': 'Price_USD'})
treasury_available = treasury_available.rename(columns={'Value': 'available_value'})
treasury_expense = treasury_expense.rename(columns={'Value': 'expense_value'})

## merge DataFrames, clean, create target

In [4]:
df = [price, transactions, fees, bond, unbonding, active_inactive, treasury_available, treasury_expense]
df_merged = reduce(lambda left, right: pd.merge(left, right, on=['Date'], how='outer'), df)

In [5]:
df_merged['Seven_day_price'] = df_merged['Price_USD'].shift(-7)
df_merged = df_merged.dropna()

In [6]:
display(df_merged.head())
display(df_merged.tail())

,Date,Price_USD,DOT_transactions,DOT_fees,Bond_value,Unbonding_value,Active_account,New_account,available_value,expense_value,Seven_day_price
1,2020-07-18,1.750289,600.0,4.2493,3.225823e+05,49257.838569,197.0,30.0,0.0,0.0,1.709566
2,2020-07-19,1.605382,666.0,9.9014,2.009450e+07,274420.000000,395.0,21.0,0.0,0.0,1.678297
3,2020-07-20,1.571536,7000.0,5.8391,2.922391e+07,18027.400000,286.0,30.0,0.0,0.0,1.702217
4,2020-07-21,1.588110,0.0,4.5909,1.489727e+05,25100.000000,173.0,18.0,0.0,0.0,1.770439
5,2020-07-22,1.570872,0.0,3.7842,2.527056e+06,800000.000000,169.0,11.0,0.0,0.0,1.771341


,Date,Price_USD,DOT_transactions,DOT_fees,Bond_value,Unbonding_value,Active_account,New_account,available_value,expense_value,Seven_day_price
634,2022-04-12,17.941162,8.573963e+06,165.978411,1.241070e+07,2.034294e+04,1917.0,2108.0,2.804684e+07,0.0,18.999490
635,2022-04-13,18.063836,1.941962e+07,147.440837,1.049751e+05,1.325278e+05,1757.0,1839.0,2.812011e+07,0.0,19.100654
636,2022-04-14,18.164701,4.887517e+06,174.040428,8.649407e+05,3.778230e+06,3006.0,2066.0,2.819367e+07,0.0,18.329985
637,2022-04-15,18.428432,1.104774e+07,138.297388,5.594405e+04,4.663664e+04,1570.0,1725.0,2.826710e+07,0.0,18.660793
638,2022-04-16,18.494261,1.978290e+06,138.533526,5.663654e+04,3.605870e+05,1524.0,1872.0,2.833720e+07,0.0,18.954922
